# ETL pipeline

## 1. Load the dataset (train and test separately)
## 2. Treat missing data uniformly (as found in data exploration step)
## 3. Save dataframe

## Load python modules

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re

## 1. Load the dataset (train and test separately)


In [2]:
train_data = './data/train.csv'
test_data = './data/test.csv'
df_train = pd.read_csv(train_data)
df_test = pd.read_csv(test_data)
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
df_test.count()

PassengerId    418
Pclass         418
Name           418
Sex            418
Age            332
SibSp          418
Parch          418
Ticket         418
Fare           417
Cabin           91
Embarked       418
dtype: int64

## 2. Treat missing data uniformly (as found in data exploration step)

- Age
- Cabin
- Embarked
- Fare (test contains one missing value)

### Age

In [5]:
df_train_LUT = df_train.groupby(['Pclass', 'Sex']).agg('median').reset_index()
df_train_LUT

,Pclass,Sex,PassengerId,Survived,Age,SibSp,Parch,Fare
0,1,female,447.0,1.0,35.0,0.0,0.0,82.66455
1,1,male,480.5,0.0,40.0,0.0,0.0,41.26250
2,2,female,439.5,1.0,28.0,0.0,0.0,22.00000
3,2,male,416.5,0.0,30.0,0.0,0.0,13.00000
4,3,female,376.0,0.5,21.5,0.0,0.0,12.47500
5,3,male,466.0,0.0,25.0,0.0,0.0,7.92500


In [6]:
df_test_LUT = df_test.groupby(['Pclass', 'Sex']).agg('median').reset_index()
df_test_LUT

,Pclass,Sex,PassengerId,Age,SibSp,Parch,Fare
0,1,female,1088.0,41.0,0.0,0.0,79.02500
1,1,male,1094.0,42.0,0.0,0.0,51.86250
2,2,female,1122.0,24.0,0.0,0.0,26.00000
3,2,male,1122.0,28.0,0.0,0.0,13.00000
4,3,female,1070.5,22.0,0.0,0.0,8.08125
5,3,male,1102.5,24.0,0.0,0.0,7.89580


In [7]:
def missing_age(row, df_LUT):
    global glbl_count
    # checks if the age row of the df is NAN
    if np.isnan(row['Age']):
        glbl_count = glbl_count + 1
        # if NAN: replace with the median of that category, taking Pclass and Sex into account
        
        median_age = df_LUT[ 
            (df_LUT['Pclass'] == row['Pclass']) & 
            (df_LUT['Sex'] == row['Sex']) 
                    ].Age.values[0]
        
        # print(median_age)
        return median_age
    else:
        return row['Age']
    # if not-NAN: take the value of age

glbl_count = 0    
df_train['Age'] = df_train.apply(lambda x: missing_age(x, df_train_LUT), axis = 1)
print(f'Replaced {glbl_count} Age entries in df_train with the grouped median.')

glbl_count = 0
df_test['Age'] = df_test.apply(lambda x: missing_age(x, df_test_LUT), axis = 1)
print(f'Replaced {glbl_count} Age entries in df_test with the grouped median.')

Replaced 177 Age entries in df_train with the grouped median.
Replaced 86 Age entries in df_test with the grouped median.


### Cabin

In [8]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [9]:
def missing_cabin(row):
    if not type(row) == str:  # the nan are float type
        return 'missing'
    pat = re.findall(r'([^\d]+)\d+', row)
    if len(pat) > 0:
        return row
    return 'missing'  # not a letter

df_train['Cabin'] = df_train['Cabin'].apply(lambda x: missing_cabin(x))
df_test['Cabin'] = df_test['Cabin'].apply(lambda x: missing_cabin(x))

In [10]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,missing,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,missing,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,missing,S


In [11]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,missing,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,missing,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,missing,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,missing,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,missing,S


In [12]:
df_test.count()

PassengerId    418
Pclass         418
Name           418
Sex            418
Age            418
SibSp          418
Parch          418
Ticket         418
Fare           417
Cabin          418
Embarked       418
dtype: int64

In [13]:
df_train.count()

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            891
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          891
Embarked       889
dtype: int64

### Embarked

In [14]:
def missing_embarked(row):
    if not type(row) == str:
        if np.isnan(row):
            return 'missing'
        else:
            return row
    else:
        return row

df_train['Embarked'] = df_train['Embarked'].apply(lambda x: missing_embarked(x))
df_test['Embarked'] = df_test['Embarked'].apply(lambda x: missing_embarked(x))

In [15]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,missing,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,missing,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,missing,S


In [16]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,missing,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,missing,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,missing,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,missing,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,missing,S


In [17]:
df_train.count()

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            891
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          891
Embarked       891
dtype: int64

In [18]:
df_test.count()

PassengerId    418
Pclass         418
Name           418
Sex            418
Age            418
SibSp          418
Parch          418
Ticket         418
Fare           417
Cabin          418
Embarked       418
dtype: int64

### Fare - test only

In [19]:
df_test[df_test['Fare'].isnull()]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,missing,S


Replace with median of the third class fare.

In [20]:
df_test_LUT_fare = df_test.groupby(['Pclass']).agg('median').reset_index()
df_test_LUT_fare

,Pclass,PassengerId,Age,SibSp,Parch,Fare
0,1,1094.0,42.0,0.0,0.0,60.0000
1,2,1122.0,27.0,0.0,0.0,15.7500
2,3,1091.5,24.0,0.0,0.0,7.8958


In [21]:
def missing_fare(row, df_LUT):
    if np.isnan(row['Fare']):
        # if NAN: replace with the median of that category, taking Pclass and Sex into account
        # print(row)
        median_fare = df_LUT[ 
            (df_LUT['Pclass'] == row['Pclass'])
                    ].Fare.values[0]
        
        # print(median_age)
        return median_fare
    else:
        return row['Fare']

df_test['Fare'] = df_test.apply(lambda x: missing_fare(x, df_test_LUT_fare), axis = 1) 
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,missing,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,missing,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,missing,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,missing,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,missing,S


In [22]:
df_test[df_test['PassengerId'] == 1044]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,7.8958,missing,S


## 3. Save dataframe

In [23]:
df_train.count()

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            891
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          891
Embarked       891
dtype: int64

In [24]:
df_test.count()

PassengerId    418
Pclass         418
Name           418
Sex            418
Age            418
SibSp          418
Parch          418
Ticket         418
Fare           418
Cabin          418
Embarked       418
dtype: int64

In [25]:
train_data = './data/train.ETL.001.csv'
test_data = './data/test.ETL.001.csv'
df_train.to_csv(train_data)
df_test.to_csv(test_data)